# Intersect catchment with SOILGRIDS soil classes
Counts the occurence of each soil class in each HRU in the model setup with pyQGIS.

In [1]:
import os
import processing # QGIS algorithm runner
from pathlib import Path
from shutil import copyfile
from datetime import datetime
from qgis.core import QgsApplication, QgsVectorLayer, QgsRasterLayer, QgsProcessingFeedback
from qgis.analysis import QgsNativeAlgorithms, QgsZonalStatistics
from utils.read_files import read_from_control, make_default_path

Application path not initialized


#### Control file handling



In [2]:
# Store the name of the 'active' file in a variable
basin = "EastRiver"
controlFile = f'control_{basin}.txt'

averaging_type = 'majority' # 'majority' or 'histogram'

#### Find location of shapefile and soil class .tif

In [3]:
# Catchment shapefile path & name
catchment_path = read_from_control(controlFile,'catchment_shp_path')
catchment_name = read_from_control(controlFile,'catchment_shp_name')

In [4]:
# Specify default path if needed
if catchment_path == 'default':
    catchment_path = make_default_path('shapefiles/catchment', controlFile) # outputs a Path()
else:
    catchment_path = Path(catchment_path) # make sure a user-specified path is a Path()

In [5]:
# Forcing shapefile path & name
soil_path = read_from_control(controlFile,'parameter_soil_domain_path')
soil_name = read_from_control(controlFile,'parameter_soil_tif_name')

In [6]:
# Specify default path if needed
if soil_path == 'default':
    soil_path = make_default_path('parameters/soilclass/2_soil_classes_domain', controlFile) # outputs a Path()
else:
    soil_path = Path(soil_path) # make sure a user-specified path is a Path()

#### Find where the intersection needs to go

In [7]:
# Intersected shapefile path and name
intersect_path = read_from_control(controlFile,'intersect_soil_path')
intersect_name = read_from_control(controlFile,'intersect_soil_name')

In [8]:
# Specify default path if needed
if intersect_path == 'default':
    intersect_path = make_default_path('shapefiles/catchment_intersection/with_soilgrids', controlFile) # outputs a Path()
else:
    intersect_path = Path(intersect_path) # make sure a user-specified path is a Path()

In [9]:
# Make the folder if it doesn't exist
intersect_path.mkdir(parents=True, exist_ok=True)

In [10]:
if averaging_type == 'majority':    
    # Find the name without extension
    catchment_base = catchment_name.replace('.shp','')
    # Loop over directory contents and copy files that match the filename of the shape
    for file in os.listdir(catchment_path):
        if catchment_base in file: # copy only the relevant files in case there are more than 1 .shp files
            
            # make the output file name
            _,ext = os.path.splitext(file)                    # extension of current file
            basefile,_ = os.path.splitext(intersect_name)     # name of the intersection file w/o extension
            newfile = basefile + ext                          # new name + old extension
            
            # copy
            copyfile(catchment_path/file, intersect_path/newfile);
else:
    # remove these files if they exist
    for file in os.listdir(intersect_path):
        if intersect_name in file:
            os.remove(intersect_path/file)

#### QGIS analysis

In [11]:
# Import all native QGIS algorithms
QgsApplication.processingRegistry().addProvider(QgsNativeAlgorithms());

Application path not initialized
Application path not initialized
Application path not initialized
Application path not initialized
Application path not initialized
Application path not initialized
Application path not initialized
Application path not initialized
Application path not initialized
Application path not initialized
Application path not initialized
Application path not initialized
Application path not initialized
Application path not initialized
Application path not initialized
Application path not initialized
Application path not initialized
Application path not initialized
Application path not initialized
Application path not initialized
Application path not initialized
Application path not initialized
Application path not initialized
Application path not initialized
Application path not initialized
Application path not initialized


In [14]:
# Convert Path() to string for QGIS
intersect_file = str(intersect_path/intersect_name)
soil_file      = str(soil_path/soil_name)

In [15]:
# Load the shape and raster
layer_polygon = QgsVectorLayer(intersect_file,'merit_hydro_basin','ogr') # this works
layer_raster  = QgsRasterLayer(soil_file,'soilgrids_soil_classes') # this works

Application path not initialized
Application path not initialized
Application path not initialized
Application path not initialized


In [16]:
# Check we loaded the layers correctly
if not layer_raster.isValid():
    print('Raster layer failed to load')
    
if not layer_polygon.isValid():
    print('Polygon layer failed to load')

In [17]:
# Algorithm feedback
feedback = QgsProcessingFeedback()

In [18]:
# Run the zonalHistogram
# Specify the parameters for the zonalHistogram function
band = 1 # raster band with the data we are after
params = { 'COLUMN_PREFIX': 'USGS_',
           'INPUT_RASTER' : layer_raster, 
           'INPUT_VECTOR' : layer_polygon, 
           'OUTPUT'       : str(intersect_path/intersect_name), 
           'RASTER_BAND'  : band }
if averaging_type == 'majority':
    # Create a zonal statistics object, automatically saved to file
    zonalstats = QgsZonalStatistics(layer_polygon,                 # shapefile
                                    layer_raster,                  # .tif
                                    'soilClass_',                       # prefix for the new column added to the shapefile  
                                    band,                          # raster band we're interested in
                                    stats=QgsZonalStatistics.Majority).calculateStatistics(None) # taking the majority of soil pixels
elif averaging_type == "histogram":
    res = processing.run("native:zonalhistogram", params, feedback=feedback)

QObject::startTimer: Timers can only be used with threads started with QThread
Warning 1: Value 206830746.98144582 of field HRU_area of feature 0 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 372989589.04101098 of field HRU_area of feature 1 not successfully written. Possibly due to too larger number with respect to field width
Warning 1: Value 163108286.0009017 of field HRU_area of feature 2 not successfully written. Possibly due to too larger number with respect to field width
